# Recommandation system by KNN 

Import the libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
Ratings = pd.read_csv("D:/CMS/CMS_2019/Projects/Recommendation_System/Recommandation by KNN/Data/ratings.csv",
                      usecols=['userId', 'movieId', 'rating'],
                      dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

Movies = pd.read_csv("D:/CMS/CMS_2019/Projects/Recommendation_System/Recommandation by KNN/Data/movies.csv",
                     usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})

In [3]:
Movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
Ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
Data = pd.merge(Movies,Ratings,on='movieId')
Data.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [8]:
combine_movie_rating = Data.dropna(axis=0,subset=['title'])
Movie_rating_count = (combine_movie_rating.
                        groupby(by = ['title'])['rating'].
                        count().reset_index().
                      rename(columns = {'rating': 'totalRatingCount'})
                      [['title', 'totalRatingCount']])
Movie_rating_count.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [9]:
Rating_with_Total_Rating_Count = combine_movie_rating.merge(Movie_rating_count,left_on='title',right_on='title',how = 'left')
Rating_with_Total_Rating_Count.head()

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [11]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(Movie_rating_count['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [12]:
popularity_threshold = 50
Rating_popular_movie= Rating_with_Total_Rating_Count.query('totalRatingCount >= @popularity_threshold')
Rating_popular_movie.head()

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.000,215
1,1,Toy Story (1995),5,4.000,215
2,1,Toy Story (1995),7,4.500,215
3,1,Toy Story (1995),15,2.500,215
4,1,Toy Story (1995),17,4.500,215


Creating the pivot table

In [15]:
Movie_features_data=Rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
Movie_features_data.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


Converting into array matrix

In [17]:
from scipy.sparse import csr_matrix

Movie_features_data_matrix = csr_matrix(Movie_features_data.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(Movie_features_data_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [19]:
Movie_features_data.shape

(450, 606)

In [20]:
query_index = np.random.choice(Movie_features_data.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(Movie_features_data.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

438


In [21]:
Movie_features_data.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [22]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(Movie_features_data.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, Movie_features_data.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Who Framed Roger Rabbit? (1988):

1: Little Mermaid, The (1989), with distance of 0.41352808475494385:
2: Indiana Jones and the Temple of Doom (1984), with distance of 0.41871482133865356:
3: Crocodile Dundee (1986), with distance of 0.4453814625740051:
4: Bug's Life, A (1998), with distance of 0.44994378089904785:
5: South Park: Bigger, Longer and Uncut (1999), with distance of 0.4548070430755615:
